In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

from PIL import Image
# from torchinfo import summary
import torch
import os
import warnings
warnings.filterwarnings("ignore")
from typing import Tuple

from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np

from datasets import load_dataset


In [2]:
print('Number CUDA Devices:', torch.cuda.device_count())
print ('Current cuda device: ', torch.cuda.current_device(), ' **May not correspond to nvidia-smi ID above, check visibility parameter')

Number CUDA Devices: 1
Current cuda device:  0  **May not correspond to nvidia-smi ID above, check visibility parameter


In [3]:
import wandb
# wandb.login("1fa58b4e42c64c2531b3abeb43c04f5991be307e")

In [11]:
# !pwd
!ls ../mnt/local/data/kalexu97/large_dataset

train


In [2]:
# dataset_folder_name = '../mnt/local/data/kalexu97/large_dataset'
dataset_folder_name = '../mnt/local/data/kalexu97/short_dataset'
dataset = load_dataset("imagefolder", data_dir=dataset_folder_name)
train_dataset, test_dataset = dataset['train'], dataset['test']


Resolving data files:   0%|          | 0/500 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/250 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [3]:
dataset


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 500
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 250
    })
})

In [19]:
test_dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 7027
})

In [4]:
from transformers import ViTImageProcessor

model_name_or_path = 'google/vit-base-patch16-224-in21k'
processor = ViTImageProcessor.from_pretrained(model_name_or_path)

In [5]:
from torchvision.transforms import RandomResizedCrop, Compose, Normalize, ToTensor, Resize

normalize = Normalize(mean=processor.image_mean, std=processor.image_std)
size = (
    processor.size["shortest_edge"]
    if "shortest_edge" in processor.size
    else (processor.size["height"], processor.size["width"])
)

_transforms = Compose([Resize(224), ToTensor(), normalize])

def transforms(examples):
    examples["pixel_values"] = [_transforms(img.convert("RGB")) for img in examples["image"]]
    del examples["image"]

    return examples

def transform(example_batch):
    inputs = processor([x for x in example_batch['image']], return_tensors='pt')

    inputs['label'] = example_batch['label']
    return inputs

In [6]:
prepared_ds_train = train_dataset.with_transform(transform)
prepared_ds_test = test_dataset.with_transform(transform)


In [7]:
def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'] for x in batch]),
        'labels': torch.tensor([x['label'] for x in batch])
    }

In [15]:

# !pip install evaluate

In [8]:
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score

import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    predictions, labels = eval_pred

    # print(predictions)
    predictions = np.argmax(predictions, axis=1)
    print(predictions)
    print(labels)
    result_accuracy = accuracy.compute(predictions=predictions, references=labels)

    result = {
             'accuracy': np.mean([result_accuracy['accuracy']]),
             'kappa': np.mean([cohen_kappa_score(labels, predictions)]),
             'f1': np.mean([f1_score(labels, predictions, average='weighted')])
             }

    print(cohen_kappa_score(labels, predictions))
    print(result)

    return result


In [45]:
pred = np.array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
 # 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 # 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
                ])
trues = np.array([0, 1, 2, 2, 0, 1, 0, 2, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0
 # 1 0 0 0 0 0 0 0 0 0 0 1 3 0 0 0 0 0 0 2 0 0 0 0 2 1 0 4 0 0 0 0 0 2 0 0 0
 # 0 0 0 0 0 2 0 0 0 0 0 0 0 2 0 0 1 2 2 2 0 0 0 0 2 2
                 ])
cohen_kappa_score(trues, pred)


0.19814241486068096

In [9]:
from transformers import ViTForImageClassification

labels = train_dataset.features['label'].names
print(labels)

model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels)
)

['0', '1', '2', '3', '4']


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
# !pip install accelerate -U


In [22]:
# pip install transformers -U

In [19]:
# model=model.to('cuda:1')

In [30]:
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
# os.environ["CUDA_VISIBLE_DEVICES"]= "1"

# torch.cuda.set_device(0)

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./vit-base",
    
    report_to="wandb",  # enable logging to W&B
    run_name="vit_with_short_ds",  # name of the W&B run (optional)
    logging_steps=10,  # how often to log to W&B
    per_device_train_batch_size=16,
    evaluation_strategy="steps",
    num_train_epochs=5,
    fp16=True,
    save_steps=10,
    eval_steps=10,
    # logging_steps=10,
    learning_rate=2e-4,
    save_total_limit=2,
    remove_unused_columns=False,
    push_to_hub=False,
    load_best_model_at_end=True,
)

In [12]:
training_args.device

device(type='cuda', index=0)

In [11]:
sample_ids = np.random.choice(len(prepared_ds_test), size=100, replace=False)
sample_ids_train = np.random.choice(len(prepared_ds_train), size=len(prepared_ds_train), replace=False)
print(sample_ids)

[128 144 242  11 132 194  12 135  46 240 147 113 212 202 239 195 145  48
 222 152   3 205 216   8  19 162 248   4  62 121 219 175 169  54 165  60
   0 249  52 237  28  49  41  23 199  29 235 151 170 126 155 217 188  79
  55 164  80  33  68  77  94  75 220 227 105 134 114 231 137   2  90  88
 133 190 207 172  91 176  73  63 241  43  92 213 161  25  95 223 125 108
 140 215 156 236 127  81 122  96 184 187]


In [12]:
val_ds = prepared_ds_test.select(sample_ids)
train_ds_shiffled = prepared_ds_train.select(sample_ids_train)

In [13]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=prepared_ds_train,
    eval_dataset=val_ds,
    tokenizer=processor,
)

In [14]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

wandb: Currently logged in as: kalexu97 (alexu97skoltech). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy,Kappa,F1
10,1.605600,1.611416,0.240000,0.022885,0.137535
20,1.619900,1.610020,0.200000,0.001373,0.073394
30,1.593600,1.566982,0.320000,0.149468,0.259019
40,1.491400,1.505186,0.320000,0.144762,0.249541
50,1.403300,1.492813,0.250000,0.069017,0.178936
60,1.408400,1.605083,0.270000,0.067331,0.188072
70,1.508500,1.502065,0.330000,0.152435,0.259511
80,1.274900,1.480381,0.340000,0.175000,0.302860
90,1.235400,1.435618,0.370000,0.204545,0.334100
100,1.120000,1.414690,0.370000,0.208940,0.354931


[4 4 4 4 4 4 4 3 4 4 4 4 4 4 3 3 4 4 4 4 4 4 3 3 4 3 4 4 4 3 4 4 3 4 4 4 3
 4 4 4 3 4 4 4 4 4 3 3 4 3 4 3 4 3 4 4 4 4 4 4 4 4 4 4 3 4 3 4 4 3 4 3 4 4
 4 3 4 4 4 3 4 4 4 4 4 4 3 4 3 4 4 3 3 4 4 4 4 3 4 4]
[2 2 4 0 2 3 0 2 0 4 2 2 4 4 4 3 2 0 4 3 0 4 4 0 0 3 4 0 1 2 4 3 3 1 3 1 0
 4 1 4 0 0 0 0 3 0 4 3 3 2 3 4 3 1 1 3 1 0 1 1 1 1 4 4 2 2 2 4 2 0 1 1 2 3
 4 3 1 3 1 1 4 0 1 4 3 0 1 4 2 2 2 4 3 4 2 1 2 1 3 3]
0.022885060426845016
{'accuracy': 0.24, 'kappa': 0.022885060426845016, 'f1': 0.1375347377857463}
[1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 2 1 1 1 1 1 1 2 2 1 1 1 1 1 1 1 1 1 1
 2 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1]
[2 2 4 0 2 3 0 2 0 4 2 2 4 4 4 3 2 0 4 3 0 4 4 0 0 3 4 0 1 2 4 3 3 1 3 1 0
 4 1 4 0 0 0 0 3 0 4 3 3 2 3 4 3 1 1 3 1 0 1 1 1 1 4 4 2 2 2 4 2 0 1 1 2 3
 4 3 1 3 1 1 4 0 1 4 3 0 1 4 2 2 2 4 3 4 2 1 2 1 3 3]
0.00137311197103962
{'accuracy': 0.2, 'kappa': 0.00137311197103962, 'f1': 0.07339449541

In [20]:
metrics = trainer.evaluate(prepared_ds['test'])
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** eval metrics *****
  epoch                   =        1.0
  eval_accuracy           =       0.32
  eval_f1                 =     0.2555
  eval_kappa              =       0.15
  eval_loss               =     1.5599
  eval_runtime            = 0:02:42.49
  eval_samples_per_second =      1.538
  eval_steps_per_second   =      0.197
